### Exercise 1
#### A)
Write a small function that computes the condition number of a matrix under the max norm 

$ cond_{inf}(M) = ||M||_{inf}||M^-1||_{inf} $

In [ ]:
from numpy import array,append

def infNorm(matrix: array) -> float:
    '''returns the inf norm of the matrix'''
	assert matrix != array([]), "The given matrix is empty"
	return max(abs(matrix)) if len(matrix.shape) == 1 else max(sum(abs(matrix)))

def computeMaxNorm(matrix: array) -> float:
    '''returns the condition number of a matrix under the max norm'''
	return infNorm(matrix)*infNorm(linalg.inv(matrix))

For three frequencies, $ \omega = {0.800, 1.146, 1.400} $ calculate the condition of numbers for the matrix $ E - \omega S $.
The right-hand-side z is given with 8 significant digits. How many significant digits could we guarantee in the solution of x if everything else were assumed exact? Why?

##### Solution:
We can calculate the values of the matrices E and S with the following code:

In [121]:
Amat = 	[
    [22.13831203, 0.16279204, 0.02353879, 0.02507880,-0.02243145,-0.02951967,-0.02401863],
    [0.16279204, 29.41831006, 0.02191543,-0.06341569, 0.02192010, 0.03284020, 0.03014052],
    [0.02353879,  0.02191543, 1.60947260,-0.01788177, 0.07075279, 0.03659182, 0.06105488],
    [0.02507880, -0.06341569,-0.01788177, 9.36187184,-0.07751218, 0.00541094,-0.10660903],
    [-0.02243145, 0.02192010, 0.07075279,-0.07751218, 0.71033323, 0.10958126, 0.12061597],
    [-0.02951967, 0.03284020, 0.03659182, 0.00541094, 0.10958126, 8.38326265, 0.06673979],
    [-0.02401863, 0.03014052, 0.06105488,-0.10660903, 0.12061597, 0.06673979, 1.15733569]];

Bmat = [
    [-0.03423002, 0.09822473,-0.00832308,-0.02524951,-0.00015116, 0.05321264, 0.01834117],
    [ 0.09822473,-0.51929354,-0.02050445, 0.10769768,-0.02394699,-0.04550922,-0.02907560],
    [-0.00832308,-0.02050445,-0.11285991, 0.04843759,-0.06732213,-0.08106876,-0.13042524],
    [-0.02524951, 0.10769768, 0.04843759,-0.10760461, 0.09008724, 0.05284246, 0.10728227],
    [-0.00015116,-0.02394699,-0.06732213, 0.09008724,-0.07596617,-0.02290627,-0.12421902],
    [ 0.05321264,-0.04550922,-0.08106876, 0.05284246,-0.02290627,-0.07399581,-0.07509467],
    [ 0.01834117,-0.02907560,-0.13042524, 0.10728227,-0.12421902,-0.07509467,-0.16777868]];

# constructs the E matrix

E = []

def extendMatrx(a: list, b: list) -> list[float, ...]:
	arr = []
	for index, item in enumerate(a):
		arr = item
		arr.extend(Bmat[index])
		yield arr

E = list(extendMatrx(Amat, Bmat))
E.extend(list(extendMatrx(Bmat, Amat)))
E = array(E)

# constructs the S matrix
def constructS():
    for x in range(14):
        yield [int(x == y) * (-1 if x > 6 else 1) for y in range(14)]

S = array(list(constructS()))

The solutions for the three frequences are given by

In [ ]:
omega = [0.800, 1,146, 1,400]

for f in omega: print(E-f*S)

Since some of the elements in the matrix A have more than 8 S.D. (Significant Digits) and the solution z is given with exactly 8 SD we can assume that the solution x should be provided with at least 8 SD to guarantee a solution with 8 SD